In [1]:
using ForwardDiff
using QuantumOptics
using LinearAlgebra

In [2]:
struct System{N,S1,S2,P1,P2}
    basis1::NLevelBasis{Int}
    ψ1s::Vector{S1}
    ψ2s::Vector{S2}
    proj1s::Vector{P1}
    proj1comps::Vector{P1}
    proj2s::Matrix{P2}
    proj2sub2::Matrix{P2}
    proj2oneof::Vector{P2}
    function System{N}() where N
        basis1 = NLevelBasis(N)
        ψ1s = [nlevelstate(basis1, i) for i in 1:N]
        S1 = eltype(ψ1s)
        ψ2s = [ψ1 ⊗ ψ1 for ψ1 in ψ1s]
        S2 = eltype(ψ2s)
        proj1s = [projector(ψ1) for ψ1 in ψ1s]
        P1 = eltype(proj1s)
        proj1full = sum(proj1s)
        proj1comps = [proj1full - proj for proj in proj1s]
        proj2s = [proj1 ⊗ proj2 for proj1 in proj1s, proj2 in proj1s]
        P2 = eltype(proj2s)
        proj2sub2 = [proj2s[i, i] + proj2s[i, j] + proj2s[j, i] + proj2s[j, j]
                     for i in 1:N, j in 1:N]
        proj2oneof = [proj1s[i] ⊗ proj1comps[i] + proj1comps[i] ⊗ proj1s[i] for i in 1:N]
        return new{N,S1,S2,P1,P2}(basis1, ψ1s, ψ2s, proj1s, proj1comps,
                                  proj2s, proj2sub2, proj2oneof)
    end
end

@inline function Rele(ia, ib, i, j, θ)
    if (i == ia && j == ia) || (i == ib && j == ib)
        return complex(cos(θ / 2))
    elseif (i == ia && j == ib) || (i == ib && j == ia)
        return complex(0, -sin(θ / 2))
    elseif i == j
        return complex(one(θ))
    else
        return complex(zero(θ))
    end
end

function R(sys::System{N}, ia, ib, θ) where N
    return Operator(sys.basis1, [Rele(ia, ib, i, j, θ) for i in 1:N, j in 1:N])
end
R2(sys::System, ia, ib, θa, θb) = R(sys, ia, ib, θa) ⊗ R(sys, ia, ib, θb)

function ratio_angle(r, ε=0)
    return 2 * asin(sqrt(r)) * (1 + ε)
end
R2_ratio(sys::System, ia, ib, r, εa, εb) =
    R2(sys, ia, ib, ratio_angle(r, εa), ratio_angle(r, εb))

R2_ratio (generic function with 1 method)

In [3]:
const sys2 = System{2}()
function fid_π(εs)
    εa, εb = εs
    ψ = R2_ratio(sys2, 1, 2, 1, εa, εb) * sys2.ψ2s[1]
    return norm(sys2.proj2s[2, 2] * ψ)^2
end

fid_π (generic function with 1 method)

In [4]:
display(fid_π([0, 0]))
display(ForwardDiff.hessian(fid_π, [0.0, 0.0]))

1.0

2×2 Matrix{Float64}:
 -4.9348        3.70051e-32
  3.70051e-32  -4.9348

In [5]:
init_state(sys::System{3}, εa, εb) =
    R2_ratio(sys, 1, 3, 1/2, εa, εb) * R2_ratio(sys, 1, 2, 1/3, εa, εb) * sys.ψ2s[1]

init_state (generic function with 1 method)

In [6]:
function fid01_12(sys::System{3}, εs)
    proj0 = sqrt(2) * sys.proj2s[1, 1] + sys.proj2oneof[1]
    εa, εb = εs
    ψ = init_state(sys, εa, εb)
    ψ = proj0 * ψ
    ψ = R2_ratio(sys, 1, 2, 1, εa, εb) * ψ
    ψ = proj0 * ψ
    ψ = R2_ratio(sys, 1, 3, 1, εa, εb) * ψ
    ψ = R2_ratio(sys, 1, 3, 1, εa, εb) * ψ
    ψ = sys.proj2sub2[1, 2] * ψ
    normalize!(ψ)
    P_odd = (norm(sys.proj2s[1, 2] * ψ)^2 + norm(sys.proj2s[2, 1] * ψ)^2) / 2
    ψ = R2_ratio(sys, 1, 2, 1 / 2, εa, εb) * ψ
    c = (norm(sys.proj2s[1, 1] * ψ)^2 + norm(sys.proj2s[2, 2] * ψ)^2) / 2
    return P_odd + c
end
function fid02_10(sys::System{3}, εs)
    proj0 = sqrt(2) * sys.proj2s[1, 1] + sys.proj2oneof[1]
    εa, εb = εs
    ψ = init_state(sys, εa, εb)
    ψ = proj0 * ψ
    ψ = R2_ratio(sys, 1, 2, 1, εa, εb) * ψ
    ψ = R2_ratio(sys, 1, 3, 1, εa, εb) * ψ
    ψ = proj0 * ψ
    ψ = sys.proj2sub2[1, 2] * ψ
    normalize!(ψ)
    P_odd = (norm(sys.proj2s[1, 2] * ψ)^2 + norm(sys.proj2s[2, 1] * ψ)^2) / 2
    ψ = R2_ratio(sys, 1, 2, 1 / 2, εa, εb) * ψ
    c = (norm(sys.proj2s[1, 1] * ψ)^2 + norm(sys.proj2s[2, 2] * ψ)^2) / 2
    return P_odd + c
end
function fid12_20(sys::System{3}, εs)
    proj0 = sqrt(2) * sys.proj2s[1, 1] + sys.proj2oneof[1]
    εa, εb = εs
    ψ = init_state(sys, εa, εb)
    ψ = R2_ratio(sys, 1, 2, 1, εa, εb) * ψ
    ψ = proj0 * ψ
    ψ = R2_ratio(sys, 1, 3, 1, εa, εb) * ψ
    ψ = proj0 * ψ
    ψ = sys.proj2sub2[1, 3] * ψ
    normalize!(ψ)
    P_odd = (norm(sys.proj2s[1, 3] * ψ)^2 + norm(sys.proj2s[3, 1] * ψ)^2) / 2
    ψ = R2_ratio(sys, 1, 3, 1 / 2, εa, εb) * ψ
    c = (norm(sys.proj2s[1, 1] * ψ)^2 + norm(sys.proj2s[3, 3] * ψ)^2) / 2
    return P_odd + c
end

fid12_20 (generic function with 1 method)

In [8]:
sys3 = System{3}()
display(fid01_12(sys3, [0, 0]))
display(ForwardDiff.hessian((x)->fid01_12(sys3, x), [0.0, 0.0]))
display(fid02_10(sys3, [0, 0]))
display(ForwardDiff.hessian((x)->fid02_10(sys3, x), [0.0, 0.0]))
display(fid12_20(sys3, [0, 0]))
display(ForwardDiff.hessian((x)->fid12_20(sys3, x), [0.0, 0.0]))

1.0

2×2 Matrix{Float64}:
 -7.22258   2.07611
  2.07611  -7.22258

0.9999999999999998

2×2 Matrix{Float64}:
 -0.61685  -0.61685
 -0.61685  -0.61685

0.9999999999999998

2×2 Matrix{Float64}:
 -10.712     -1.41332
  -1.41332  -10.712